In [1]:
import findspark
findspark.init()

In [11]:
import pyspark
from pyspark.sql import SparkSession
 
spark = SparkSession.builder.getOrCreate()
 
# dict = {"name":["a", "b", "c"], "age": [1,2,3]}
dict_v = [("aas;d",[1]), ("bas", []), ("cat", []), ("des", [1,2])]
# df = spark.createDataFrame(dict, ["name", "age"])
 
# df.show()

In [12]:
# Defining the schema
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, ArrayType
df_schema = StructType([StructField("name", StringType(), True), StructField("age", ArrayType(StringType()), True )])
# DDL = "name string, age Array"
df = spark.createDataFrame(dict_v, schema=df_schema)

In [14]:
df.show()

+-----+------+
| name|   age|
+-----+------+
|aas;d|   [1]|
|  bas|    []|
|  cat|    []|
|  des|[1, 2]|
+-----+------+



In [18]:
# df.filter(df.name.contains(';')).show()
df.filter("name like '%;%'").show(20, False)

+-----+---+
|name |age|
+-----+---+
|aas;d|[1]|
+-----+---+



In [10]:
from pyspark.sql.functions import *
df.filter(size(df.age)==0).count()

2

In [12]:
from pyspark.sql.functions import udf
def test_fun(v):
  return v*2

In [13]:
sum_cols = udf(test_fun, IntegerType())
# To use in SparkSQL
# spark.udf.register("convertUDF", test_fun,StringType())

In [15]:
df.select(sum_cols("age")).show()

+-------------+
|test_fun(age)|
+-------------+
|            2|
|            4|
|            6|
|            2|
+-------------+



In [17]:
# It depends on PyArrow
import pandas as pd
from pyspark.sql.functions import pandas_udf
# Create pandas_udf()
@pandas_udf(StringType())
def to_upper(s: pd.Series) -> pd.Series:
    return s.str.upper()

ImportError: PyArrow >= 0.15.1 must be installed; however, it was not found.

In [18]:
pip install PyArrow

                                              0.0/20.6 MB ? eta -:--:--
Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Python39\lib\site-packages\pip\_vendor\urllib3\response.py", line 435, in _error_catcher
    yield
  File "C:\Python39\lib\site-packages\pip\_vendor\urllib3\response.py", line 516, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Python39\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Python39\lib\http\client.py", line 458, in read
    n = self.readinto(b)
  File "C:\Python39\lib\http\client.py", line 502, in readinto
    n = self.fp.readinto(b)
  File "C:\Python39\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Python39\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Python39\lib\ssl.py", line 1099, in read
    return self._sslobj.read(len, buffer)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Durin

In [20]:
df.printSchema()
df.schema


root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



False

In [23]:
# ColRegex
df.select(df.colRegex("`(name)*`")).show()

+----+
|name|
+----+
|aasd|
| bas|
| cat|
+----+



In [25]:
# ColRegex
df.select("name").show()

+----+
|name|
+----+
|aasd|
| bas|
| cat|
+----+



In [26]:
df.createOrReplaceTempView("df")

In [27]:
spark.sql("select * from df").show()

+----+---+
|name|age|
+----+---+
|aasd|  1|
| bas|  2|
| cat|  3|
+----+---+



In [32]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *
df.withColumn("row_num", row_number().over(Window.partitionBy("age").orderBy("age"))).show()

+----+---+-------+
|name|age|row_num|
+----+---+-------+
|aasd|  1|      1|
| des|  1|      2|
| cat|  3|      1|
| bas|  2|      1|
+----+---+-------+



In [37]:
# from pyspark.sql.functions
df.select("*",row_number().over(Window.partitionBy("age").orderBy("age")).alias("row_num"),count("age").over(Window.partitionBy("age").orderBy("age")).alias("count")).show()

+----+---+-------+-----+
|name|age|row_num|count|
+----+---+-------+-----+
|aasd|  1|      1|    2|
| des|  1|      2|    2|
| cat|  3|      1|    1|
| bas|  2|      1|    1|
+----+---+-------+-----+



In [1]:
import logging

logger = logging.getLogger("Test")
logger.setLevel(logging.INFO)
console = logging.StreamHandler()
# console.setLevel(logging.INFO)
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S%p')
# console.setFormatter(formatter)
# logger.addHandler(console)
# logger.addHandler("stdoutut")
# logger.info("Tesh")

In [34]:
from pyspark.sql.functions import spark_partition_id
# df.select(spark_partition_id).show()
# help(spark_partition_id)
# df.repartition(1)
df_partition = df.select(*df.columns, spark_partition_id().alias("pid")).show()

+----+---+---+
|name|age|pid|
+----+---+---+
| bas|  2|  0|
| cat|  3|  0|
|aasd|  1|  1|
| des|  1|  2|
+----+---+---+



In [32]:
df.rdd.getNumPartitions()

3

In [35]:
# df=df.coalesce()
# df.repartition()
help(df.repartition)

Help on method repartition in module pyspark.sql.dataframe:

repartition(numPartitions, *cols) method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` partitioned by the given partitioning expressions. The
    resulting :class:`DataFrame` is hash partitioned.
    
    :param numPartitions:
        can be an int to specify the target number of partitions or a Column.
        If it is a Column, it will be used as the first partitioning column. If not specified,
        the default number of partitions is used.
    
    .. versionchanged:: 1.6
       Added optional arguments to specify the partitioning columns. Also made numPartitions
       optional if partitioning columns are specified.
    
    >>> df.repartition(10).rdd.getNumPartitions()
    10
    >>> data = df.union(df).repartition("age")
    >>> data.show()
    +---+-----+
    |age| name|
    +---+-----+
    |  5|  Bob|
    |  5|  Bob|
    |  2|Alice|
    |  2|Alice|
    +---+-----+
    >>> data = d

Help on tuple object:

class tuple(object)
 |  tuple(iterable=(), /)
 |  
 |  Built-in immutable sequence.
 |  
 |  If no argument is given, the constructor returns an empty tuple.
 |  If iterable is specified the tuple is initialized from iterable's items.
 |  
 |  If the argument is a tuple, the return value is the same object.
 |  
 |  Built-in subclasses:
 |      asyncgen_hooks
 |      UnraisableHookArgs
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __getnewargs__(self, /)
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __iter__(self, /)
 |